In [1]:
def reliefF(X, y, k_neighbors=3):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    class_probs = np.bincount(y) / len(y)
    max_val = np.max(X, axis=0)
    min_val = np.min(X, axis=0)
    
    m = n_samples
    
    selected_indices = []
    
    for i in range(m):
        valid_indices = np.setdiff1d(np.arange(n_samples), selected_indices)
        
        random_index = np.random.choice(valid_indices)
        
        selected_indices.append(random_index)
        
        distances = np.sum(np.abs(X - X[random_index]), axis=1)
        
        nearest_indices = np.argsort(distances)
        
        nearest_same_class = nearest_indices[(y[nearest_indices] == y[random_index]) & (nearest_indices != random_index)][:k_neighbors]
        
        nearest_diff_class = nearest_indices[(y[nearest_indices] != y[random_index])][:k_neighbors]
        
        for j in range(n_features):
            nearest_same_dist = distances[nearest_same_class]
            maxmin = (max_val[j] - min_val[j]) if (max_val[j] - min_val[j]) != 0 else 1
            weights_same = (np.abs(X[random_index, j] - X[nearest_same_class, j]) / maxmin)
            weights[j] -= (np.sum(weights_same) / (k_neighbors * m))
            
            weights_diff = 0
            for C, class_prob in enumerate(class_probs):
                if C != y[random_index]:
                    miss_prob = (class_prob / (1 - class_probs[y[random_index]]))
                    nearest_diff_dist = distances[nearest_diff_class]
                    miss_weight = np.abs(X[random_index, j] - X[nearest_diff_class, j])
                    weights_diff += (miss_prob * np.sum(miss_weight) / (k_neighbors * m))
            
            weights[j] += (weights_diff / (k_neighbors * m))
    
    return weights


In [2]:
import pandas as pd
import numpy as np
import time

from sklearn.preprocessing import StandardScaler
from skrebate import ReliefF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv('bodyPerformance.csv')

# df = df.iloc[:, 1:]
# df.fillna(df.mode().iloc[0], inplace=True) # Replace NaN values with mode of corresponding column
# use the factorize() function to convert the column of strings to integers
# non_int_columns = df.select_dtypes(include=['object']).columns.tolist()
# for i in non_int_columns:
#     df[i] = pd.factorize(df[i])[0] + 1
    
# Delete duplicate rows
df = df.drop_duplicates()
len_row, len_col = df.shape

In [3]:
print(len_row, len_col)

13392 12


In [4]:
label_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

df['class'] = df['class'].replace(label_mapping)

label_mapping = {'M': 1, 'F': 2}

df['gender'] = df['gender'].replace(label_mapping)

In [5]:
df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,2
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,2
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1


In [27]:
# df = df.drop(['stdevy135','korelasi135'], axis=1)

In [6]:
# Membaca data
X = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1])

# X_col = df.iloc[:, :-1].columns.tolist()

# # Create a StandardScaler object
# scaler = StandardScaler()

# # Fit the scaler to the dataset
# scaler.fit(X)

# # Transform the dataset using the scaler
# X_std = scaler.transform(X)

# df_X_std = pd.DataFrame(data=X_std, columns=X_col)

In [7]:
def reliefF(X, y, k_neighbors=3):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    class_probs = np.bincount(y) / len(y)
    max_val = np.max(X, axis=0)
    min_val = np.min(X, axis=0)
    
    m = n_samples
    
    selected_indices = []
    
    for i in range(m):
        valid_indices = np.setdiff1d(np.arange(n_samples), selected_indices)
        
        random_index = np.random.choice(valid_indices)
        
        selected_indices.append(random_index)
        
        distances = np.sum(np.abs(X - X[random_index]), axis=1)
        
        nearest_indices = np.argsort(distances)
        
        nearest_same_class = nearest_indices[(y[nearest_indices] == y[random_index]) & (nearest_indices != random_index)][:k_neighbors]
        
        nearest_diff_class = nearest_indices[(y[nearest_indices] != y[random_index])][:k_neighbors]
        
        for j in range(n_features):
            nearest_same_dist = distances[nearest_same_class]
            maxmin = (max_val[j] - min_val[j]) if (max_val[j] - min_val[j]) != 0 else 1
            weights_same = np.abs(X[random_index, j] - X[nearest_same_class, j]) / maxmin
            weights[j] -= np.sum(weights_same) / (k_neighbors * m)
            
            weights_diff = 0
            for C, class_prob in enumerate(class_probs):
                if C != y[random_index]:
                    miss_prob = class_prob / (1 - class_probs[y[random_index]])
                    nearest_diff_dist = distances[nearest_diff_class]
                    miss_weight = np.abs(X[random_index, j] - X[nearest_diff_class, j])
                    weights_diff += miss_prob * np.sum(miss_weight) / (k_neighbors * m)
            
            weights[j] += weights_diff / (k_neighbors * m)
    
    return weights


In [12]:
# Assuming X_std and y are defined
feature_weights = reliefF(X, y)

combines = {'Features': df.iloc[:, :-1].columns, 'Weights': feature_weights}
weight_f = pd.DataFrame(combines)
weight_f = weight_f.sort_values('Weights', ascending=False)


KeyboardInterrupt: 

In [13]:
weight_f = weight_f.sort_values('Weights', ascending=False)
weight_f.head(100)

,Features,Weights
8,sit and bend forward_cm,-0.013687
10,broad jump_cm,-0.014212
1,gender,-0.019040
6,systolic,-0.019829
5,diastolic,-0.024486
3,weight_kg,-0.030994
4,body fat_%,-0.038340
2,height_cm,-0.044112
7,gripForce,-0.045157
9,sit-ups counts,-0.045495


In [12]:
list_feat = weight_f['Features'].tolist()
list_feat[::-1]
X_train, X_test, y_train, y_test = train_test_split(df_X_std, y, test_size = 0.3, random_state = 42)

accs = {}
timer = {}
for num_feat in range(len_col-1):
    X_selected = X_train[list_feat[:num_feat+1]]
    
    # Train a random forest model on the training data
    model = RandomForestClassifier(random_state = 42)
    start_time = time.time()
    model.fit(X_selected, y_train)
    end_time = time.time()

    # Evaluate the model on the testing data
    X_selected_test = X_test[list_feat[:num_feat+1]]
    y_pred = model.predict(X_selected_test)
    acc = accuracy_score(y_test, y_pred)
    
    times = end_time - start_time
    
    timer[f'k = {num_feat+1}'] = times
    accs[f'k = {num_feat+1}'] = acc

In [13]:
accs

{'k = 1': 0.4,
 'k = 2': 0.2,
 'k = 3': 0.23333333333333334,
 'k = 4': 0.3,
 'k = 5': 0.3333333333333333,
 'k = 6': 0.36666666666666664,
 'k = 7': 0.26666666666666666,
 'k = 8': 0.36666666666666664,
 'k = 9': 0.36666666666666664,
 'k = 10': 0.43333333333333335,
 'k = 11': 0.3333333333333333,
 'k = 12': 0.36666666666666664,
 'k = 13': 0.3333333333333333,
 'k = 14': 0.3333333333333333,
 'k = 15': 0.3,
 'k = 16': 0.3,
 'k = 17': 0.36666666666666664,
 'k = 18': 0.36666666666666664,
 'k = 19': 0.26666666666666666,
 'k = 20': 0.26666666666666666,
 'k = 21': 0.3,
 'k = 22': 0.3333333333333333,
 'k = 23': 0.36666666666666664,
 'k = 24': 0.43333333333333335,
 'k = 25': 0.36666666666666664,
 'k = 26': 0.26666666666666666,
 'k = 27': 0.36666666666666664,
 'k = 28': 0.3333333333333333,
 'k = 29': 0.3333333333333333,
 'k = 30': 0.3333333333333333,
 'k = 31': 0.3333333333333333,
 'k = 32': 0.3,
 'k = 33': 0.3333333333333333,
 'k = 34': 0.3,
 'k = 35': 0.26666666666666666}

In [14]:
print(f"Accuracy using all columns= {accs[f'k = {len_col-1}']}")
print(f"Elapsed Time = {timer[f'k = {len_col-1}']}")
print()
print(f"Accuracy using 3 best columns= {accs[f'k = 3']}")
print(f"Elapsed Time = {timer[f'k = 3']}")
print()
print(f"Accuracy using 10 best columns= {accs[f'k = 10']}")
print(f"Elapsed Time = {timer[f'k = 10']}")
print()
best_k = max(accs, key=accs.get)
print(f"Best {best_k} with Accuracy = {accs[best_k]}")
print(f"Elapsed Time = {timer[best_k]}")
print()

Accuracy using all columns= 0.26666666666666666
Elapsed Time = 0.18170905113220215

Accuracy using 3 best columns= 0.23333333333333334
Elapsed Time = 0.16551470756530762

Accuracy using 10 best columns= 0.43333333333333335
Elapsed Time = 0.17218589782714844

Best k = 10 with Accuracy = 0.43333333333333335
Elapsed Time = 0.17218589782714844



In [19]:
worsts = [3,10]
for worst in worsts:
    # USING 3 Worst columns & USING 10 Worst columns
    X_select = X_selected[list_feat[-worst:]]

    
    # Train a random forest model on the training data
    model = RandomForestClassifier(random_state = 42)
    start_time = time.time()
    model.fit(X_select, y_train)
    end_time = time.time()

    # Evaluate the model on the testing data
    X_select_test = X_selected_test[list_feat[-worst:]]
    y_pred = model.predict(X_select_test)
    acc = accuracy_score(y_test, y_pred)

    times = end_time - start_time

    print(f"Accuracy using {worst} worst columns= {acc}")
    print(f"Elapsed Time = {times}")
    print()

Accuracy using 3 worst columns= 0.26666666666666666
Elapsed Time = 0.23040533065795898

Accuracy using 10 worst columns= 0.36666666666666664
Elapsed Time = 0.18795013427734375

